# Target: Using BulkRNABert (the pre-trained model), provide it with the TCGA-BRCA gene expression data, to fine-tune/transfer learning it to be a domain model. [Please correct me if anything is wrong/inaccurate.]

# ------------- Strategie/Key Points -----------------
## 1. Get the data: 
### 1) BulkRNABert pre-trained on RNA-seq data (expression value in TPM), so our TCGA data should be in TPM unit as well.
### 2) common_id.txt is provided in official repo of BulkRNABert, which gives the order and lists of 19062 genes that should be used when feed the model. We will use this file to filter out these genes from our data and lay them out according to the common_id.txt file's order. (You can also choose to use gene_name, which takes 1 extra step to transform id -> name)

### 3) The sample-subtype data can be found as well, use this data to bind the subtype, whose sample ID is also in our expression data, to the gene expression data's column. 


# I already have the sample x (gene+subtype) TPM data. I now will filter out commom_genes as the data prep for model training

In [23]:
# Extracting common genes from the model repo (Model = BulkRNABert)

import pandas as pd
import numpy as np

# 1. Load the common gene IDs (The master list)
with open('common_gene_id.txt', 'r') as f:
    ordered_common_genes = [line.strip() for line in f if line.strip()]

# 2. Load your TPM TSV file (already log-transformed)
df = pd.read_csv('BRCA-data_transformer.tsv', sep='\t', index_col=0)

# 3. Handle the Subtype column separately
subtype_col_name = df.columns[-1]
subtypes = df[subtype_col_name]

# 4. Extract only the genes that exist in your data
existing_genes = [g for g in ordered_common_genes if g in df.columns]
df_filtered = df[existing_genes]

# 5. ADD MISSING GENES (Zero-Padding)
# Identify which genes from the master list are missing in your data
missing_genes = [g for g in ordered_common_genes if g not in df.columns]

if missing_genes:
    print(f"Warning: {len(missing_genes)} genes were missing from your data. Adding them as 0.0")
    # Create a dataframe of zeros for the missing genes
    zero_data = np.zeros((len(df_filtered), len(missing_genes)))
    df_zeros = pd.DataFrame(zero_data, index=df_filtered.index, columns=missing_genes)
    
    # Combine existing data with the zero-filled columns
    df_combined = pd.concat([df_filtered, df_zeros], axis=1)
else:
    df_combined = df_filtered

# 6. ENFORCE FINAL ORDER
# This step is critical: it rearranges columns to match 'common_gene_id.txt' exactly

print(f"Total lines in file: {len(ordered_common_genes)}")
print(f"Unique gene names: {len(set(ordered_common_genes))}")
print(f"Number of duplicates: {len(ordered_common_genes) - len(set(ordered_common_genes))}")

df_final = df_combined[ordered_common_genes].copy()
print(df_final.shape)

# 7. Re-attach Subtype and Save
#df_final[subtype_col_name] = subtypes
# df_final.to_csv('ready_for_bulkRNABert.csv')

print(f"Final file created with {len(ordered_common_genes)} genes + Subtype.")

Total lines in file: 19062
Unique gene names: 19062
Number of duplicates: 0
(991, 19062)
Final file created with 19062 genes + Subtype.


# EDA of prepared dataset

In [1]:
# Exploratory Data Analysis (EDA) on the prepared dataset

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA

def perform_full_eda(file_path):
    # 1. Load the dataset
    # sep='\t' if your file is the TSV version, else use default for CSV
    print(f"--- Loading file: {file_path} ---")
    df = pd.read_csv(file_path, index_col=0)
    
    # Separate features (X) and labels (y)
    # Assuming the last column is 'Subtype'
    X = df.iloc[:, :-1]
    y = df.iloc[:, -1]
    
    # 2. Data Integrity and Summary Statistics
    print("\n[STEP 1] Data Structure Overview:")
    print(f"- Total Samples: {df.shape[0]}")
    print(f"- Total Genes: {df.shape[1] - 1}")
    print(f"- Missing Values: {df.isnull().sum().sum()}")
    
    print("\n[STEP 2] Subtype Distribution:")
    print(y.value_counts())

    # Create a layout for visualizations
    fig, axes = plt.subplots(2, 2, figsize=(20, 12))

    # 3. Boxplot: Check expression ranges for the first 20 genes
    # Useful for verifying Log2 transformation and scale
    sns.boxplot(data=X.iloc[:, :20], ax=axes[0, 0])
    axes[0, 0].set_title("Expression Distribution (First 20 Genes)")
    axes[0, 0].tick_params(axis='x', rotation=45)

    # 4. PCA: Visualize how well subtypes separate in 2D space
    # This is the most critical step for classification potential
    print("\n[STEP 3] Computing PCA...")
    pca = PCA(n_components=2)
    X_pca = pca.fit_transform(X)
    pca_df = pd.DataFrame(X_pca, columns=['PC1', 'PC2'])
    pca_df['Subtype'] = y.values

    sns.scatterplot(x='PC1', y='PC2', hue='Subtype', data=pca_df, 
                    palette='viridis', alpha=0.6, ax=axes[0, 1])
    explained_var = np.sum(pca.explained_variance_ratio_) * 100
    axes[0, 1].set_title(f"PCA: Total Explained Variance {explained_var:.2f}%")

    # 5. Correlation Heatmap: Check co-expression patterns
    # Using first 50 genes to visualize biological gene-gene relationships
    corr = X.iloc[:, :50].corr()
    sns.heatmap(corr, cmap='coolwarm', xticklabels=False, yticklabels=False, ax=axes[1, 0])
    axes[1, 0].set_title("Gene Correlation Heatmap (Top 50 Genes)")

    # 6. Mean Expression Histogram: Check for sparsity or bias
    # Shows the average expression profile across all genes
    sns.histplot(X.mean(axis=0), kde=True, color='skyblue', ax=axes[1, 1])
    axes[1, 1].set_title("Distribution of Gene Expression Means")
    axes[1, 1].set_xlabel("Average Log2 Expression")

    plt.tight_layout()
    plt.show()

# To execute: replace with your actual processed filename
perform_full_eda("ready_for_bulkRNABert.csv")

ModuleNotFoundError: No module named 'seaborn'

# Need to split out the training set and testing set

In [1]:
import pandas as pd

df = pd.read_csv("ready_for_bulkRNABert.csv", index_col=0)

df.shape
df.head()


,TSPAN6,TNMD,DPM1,SCYL3,C1orf112,FGR,CFH,FUCA2,GCLC,NFYA,...,MIR4723,MIR6748,LINC01226,MIR1302-10,MIR3064,MIR6787,MIR943,MIR6785,MIR4467,Subtype
TCGA-AQ-A0Y5-01,2.132741,0.146003,5.284151,2.428544,1.158337,1.088209,3.229680,4.749003,2.424546,3.951205,...,0.0,0.0,0.157432,0.0,0.0,0.0,0.0,0.0,0.0,BRCA_LumA
TCGA-C8-A274-01,4.768010,0.000000,5.051694,3.356299,2.458172,0.981926,1.595074,3.433373,1.924062,4.234601,...,0.0,0.0,0.005903,0.0,0.0,0.0,0.0,0.0,0.0,BRCA_LumB
TCGA-BH-A0BD-01,3.049735,1.071763,4.618797,2.680774,2.494236,1.638027,3.022172,3.817460,2.038647,3.883425,...,0.0,0.0,0.025454,0.0,0.0,0.0,0.0,0.0,0.0,BRCA_LumB
TCGA-B6-A1KC-01,3.009311,0.185993,4.852973,2.381671,1.565841,0.783079,2.527145,3.926834,2.106181,4.635743,...,0.0,0.0,0.060462,0.0,0.0,0.0,0.0,0.0,0.0,BRCA_LumB
TCGA-AO-A0J5-01,2.929999,0.467071,3.958935,2.787265,1.221939,1.515965,3.769169,4.124882,1.845470,3.666961,...,0.0,0.0,0.004034,0.0,0.0,0.0,0.0,0.0,0.0,BRCA_LumA


In [3]:
X = df.iloc[:, :-1]   # All gene expr
y = df.iloc[:, -1]    # subtype labels

print(X.shape)   # (n_samples, n_genes)
print(y.shape)   # (n_samples,)
print(y.value_counts())


(991, 19918)
(991,)
Subtype
BRCA_LumA      504
BRCA_LumB      197
BRCA_Basal     176
BRCA_Her2       78
BRCA_Normal     36
Name: count, dtype: int64


In [10]:
# Split Training/Testing datasets:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    stratify=y,
    random_state=42
)

print(X_train.shape, X_test.shape)
print(y_train.value_counts())
print(y_test.value_counts())


(792, 19918) (199, 19918)
Subtype
BRCA_LumA      403
BRCA_LumB      157
BRCA_Basal     141
BRCA_Her2       62
BRCA_Normal     29
Name: count, dtype: int64
Subtype
BRCA_LumA      101
BRCA_LumB       40
BRCA_Basal      35
BRCA_Her2       16
BRCA_Normal      7
Name: count, dtype: int64


In [11]:
# sklearn baseline

import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

# 1) baseline: 标准化 + 多分类逻辑回归（稳定、快）
clf = Pipeline([
    ("scaler", StandardScaler(with_mean=False)),  # 高维稀疏/大矩阵更安全
    ("lr", LogisticRegression(max_iter=5000, n_jobs=-1, class_weight="balanced"))
])

clf.fit(X_train, y_train)

pred = clf.predict(X_test)
print(classification_report(y_test, pred, digits=4))
print(confusion_matrix(y_test, pred))


              precision    recall  f1-score   support

  BRCA_Basal     1.0000    0.9714    0.9855        35
   BRCA_Her2     0.8667    0.8125    0.8387        16
   BRCA_LumA     0.9118    0.9208    0.9163       101
   BRCA_LumB     0.7442    0.8000    0.7711        40
 BRCA_Normal     0.8000    0.5714    0.6667         7

    accuracy                         0.8844       199
   macro avg     0.8645    0.8152    0.8356       199
weighted avg     0.8860    0.8844    0.8842       199

[[34  1  0  0  0]
 [ 0 13  0  3  0]
 [ 0  0 93  7  1]
 [ 0  1  7 32  0]
 [ 0  0  2  1  4]]


In [12]:
# Keep all the settings for repeatitiion
import pandas as pd
import os

outdir = "splits_v1"
os.makedirs(outdir, exist_ok=True)

X_train.to_csv(f"{outdir}/X_train.csv")
X_test.to_csv(f"{outdir}/X_test.csv")
y_train.to_csv(f"{outdir}/y_train.csv", header=True)
y_test.to_csv(f"{outdir}/y_test.csv", header=True)


In [13]:
# Prepare for Embedding: merge into single table (train/test)
train_df = X_train.copy()
train_df["Subtype"] = y_train.values
test_df = X_test.copy()
test_df["Subtype"] = y_test.values

train_df.to_csv(f"{outdir}/train_table.csv")
test_df.to_csv(f"{outdir}/test_table.csv")


# Input to BulkRNABert
model id = bulk_rna_bert_tcga

In [14]:
import pandas as pd

train_df = pd.read_csv("splits_v1/train_table.csv", index_col=0)
test_df  = pd.read_csv("splits_v1/test_table.csv",  index_col=0)

y_train = train_df["Subtype"]
y_test  = test_df["Subtype"]

X_train = train_df.drop(columns=["Subtype"])
X_test  = test_df.drop(columns=["Subtype"])


In [17]:
# Loading BulkRNABert
import torch
from transformers import AutoConfig, AutoModel, AutoTokenizer

device = "cuda" if torch.cuda.is_available() else "cpu"

config = AutoConfig.from_pretrained(
    "InstaDeepAI/BulkRNABert",
    trust_remote_code=True,
)
config.embeddings_layers_to_save = (4,)

tokenizer = AutoTokenizer.from_pretrained(
    "InstaDeepAI/BulkRNABert",
    trust_remote_code=True,
)

model = AutoModel.from_pretrained(
    "InstaDeepAI/BulkRNABert",
    config=config,
    trust_remote_code=True,
).to(device).eval()

print("device:", device)
print("n_genes:", config.n_genes)


config.json:   0%|          | 0.00/575 [00:00<?, ?B/s]

D:\anaconda3\envs\hackathon\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\cy03c\.cache\huggingface\hub\models--InstaDeepAI--BulkRNABert. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


bulkrnabert.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/InstaDeepAI/BulkRNABert:
- bulkrnabert.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


tokenizer_config.json:   0%|          | 0.00/392 [00:00<?, ?B/s]

...

A new version of the following files was downloaded from https://huggingface.co/InstaDeepAI/BulkRNABert:
- tokenizer.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


special_tokens_map.json:   0%|          | 0.00/3.00 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/24.0M [00:00<?, ?B/s]

device: cuda
n_genes: 19062


In [ ]:
print(model)